<a href="https://colab.research.google.com/github/jonathan-root/SAFE/blob/main/Copia_de_SAFE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Librerias

In [ ]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, r2_score, confusion_matrix, mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# LECTURA Y VISUALIZACIÓN DE LOS DATOS

In [ ]:
# Identifica los valores nulos
def count_nulls(df):
    null_counts = df.isnull().sum()
    return null_counts

# Identifica los valores nulos codificados como cadenas en un dataframe.
def identify_string_nulls(df):
    # Define una lista de cadenas que se considerarán como valores nulos
    null_set = {'NULL', 'NAN', 'NA', 'NAT', 'NaN', 'nAn', 'NaT', 'nAt'}
    
    # Itera sobre cada fila del dataframe y busca los valores nulos codificados como cadenas
    for i, row in df.iterrows():
        for col in df.columns:
            if pd.isna(row[col]):
                continue
            elif type(row[col]) == str and row[col].strip().upper() in null_set:
                row[col] = np.nan
        df.iloc[i, :] = row
    
    return df

In [ ]:
# Cargar los datos
h1 = pd.read_csv("H1.csv")
h2 = pd.read_csv("H2.csv")

In [ ]:
h1.head()

In [ ]:
h1.info()

In [ ]:
# Checamos si tienen datos nulos o vacios
count_nulls(h1)

In [ ]:
# Checamos si tienen datos nulos o vacios pero tambien con los que son de tipo string
count_nulls(identify_string_nulls(h1))

In [ ]:
h2.head()

In [ ]:
h2.info()

In [ ]:
# Checamos si tienen datos nulos o vacios
count_nulls(h2)

In [ ]:
# Checamos si tienen datos nulos o vacios pero tambien con los que son de tipo string
count_nulls(identify_string_nulls(h2))

In [ ]:
# Reemplazar valores vacíos o nulos a "0"
h1.fillna(0, inplace=True)
h2.fillna(0, inplace=True)

In [ ]:
# Crear una lista con los DataFrames y sus respectivos nombres
dataframes = [(h1, 'Hotel 1'), (h2, 'Hotel 2')]

# Verificar si hay datos nulos o vacíos en cada DataFrame
for df, name in dataframes:
    if df.isnull().values.any():
        print(f"Hay datos nulos o vacíos en el archivo {name}.")
    else:
        print(f"No hay datos nulos o vacíos en el archivo {name}.")

In [ ]:
# Conversión de variables categóricas en numéricas
h1 = pd.get_dummies(h1)
h2 = pd.get_dummies(h2)

# Evaluación con el modelo de logística vainilla

Evaluación de la regresión logística vainilla.

Dividir los datos en conjuntos de entrenamiento y prueba

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(h1.drop('IsCanceled', axis=1), h1['IsCanceled'], test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(h2.drop('IsCanceled', axis=1), h2['IsCanceled'], test_size=0.2, random_state=42)

Crear un modelo de regresión logística vainilla y ajustarlo a los datos de entrenamiento

In [ ]:
logreg_H1 = LogisticRegression(solver='lbfgs')
logreg_H1 = logreg_H1.fit(X_train1, y_train1)

In [ ]:
logreg_H2 = LogisticRegression(solver='lbfgs')
logreg_H2 = logreg_H2.fit(X_train2, y_train2)

In [ ]:
# Evaluar el modelo en los datos de prueba
logreg_y_pred1 = logreg_H1.predict(X_test1)
logreg_y_pred2 = logreg_H2.predict(X_test2)

In [ ]:
# Calcular el coeficiente de determinación del modelo
print("Coeficiente de determinación del modelo regresión lineal en H1: %0.20f" % r2_score(y_test1, logreg_y_pred1))
print("Coeficiente de determinación del modelo regresión lineal en H2: %0.20f" % r2_score(y_test2, logreg_y_pred2))

In [ ]:
# graficar la línea de regresión y los puntos de datos
plt.scatter(y_test1, logreg_y_pred1)
plt.plot([y_test1.min(), y_test1.max()], [y_test1.min(), y_test1.max()], 'k--', lw=2)
plt.xlabel('Valor real')
plt.ylabel('Predicción')
plt.title('Precisión del modelo lineal (R1 = %0.20f)' % r2_score(y_test1, logreg_y_pred1))
plt.show()

In [ ]:
# graficar la línea de regresión y los puntos de datos
plt.scatter(y_test2, logreg_y_pred2)
plt.plot([y_test2.min(), y_test2.max()], [y_test2.min(), y_test2.max()], 'k--', lw=2)
plt.xlabel('Valor real')
plt.ylabel('Predicción')
plt.title('Precisión del modelo lineal (R2 = %0.20f)' % r2_score(y_test2, logreg_y_pred2))
plt.show()

In [ ]:
# Calculando el MSE
print("MSE (Error cuadrático medio) del modelo regresión lineal en H1: %0.20f" % mean_squared_error(y_test1, logreg_y_pred1))
print("MSE (Error cuadrático medio) del modelo regresión lineal en H2: %0.20f" % mean_squared_error(y_test2, logreg_y_pred2))

In [ ]:
# calcular el MSE para diferentes tamaños del conjunto de datos de prueba
mse_values_logreg = []
sizes_logreg = range(1, len(y_test1) + 1)
for s in sizes_logreg:
    mse_values_logreg.append(mean_squared_error(y_test1[:s], logreg_y_pred1[:s]))

# graficar el MSE
plt.plot(sizes_logreg, mse_values_logreg)
plt.title('MSE del modelo lineal H1')
plt.xlabel('Tamaño del conjunto de prueba')
plt.ylabel('MSE')
plt.show()

In [ ]:
# calcular el MSE para diferentes tamaños del conjunto de datos de prueba
mse_values_logreg = []
sizes_logreg = range(1, len(y_test2) + 1)
for s in sizes_logreg:
    mse_values_logreg.append(mean_squared_error(y_test2[:s], logreg_y_pred2[:s]))

# graficar el MSE
plt.plot(sizes_logreg, mse_values_logreg)
plt.title('MSE del modelo lineal H2')
plt.xlabel('Tamaño del conjunto de prueba')
plt.ylabel('MSE')
plt.show()

In [ ]:
# Calcular la precisión del modelo
print("Precisión del modelo Vainilla en H1: %0.20f" % accuracy_score(y_test1, logreg_y_pred1))
print("Precisión del modelo Vainilla en H2: %0.20f" % accuracy_score(y_test2, logreg_y_pred2))

# EVALUACIÓN CON EL MODELO LINEAL

In [ ]:
# Evaluar el modelo lineal
# Crear un modelo de regresión lineal y ajustarlo a los datos de entrenamiento
linreg_H1 = LinearRegression()
linreg_H1 = linreg_H1.fit(X_train1, y_train1)

In [ ]:
linreg_H2 = LinearRegression()
linreg_H2 = linreg_H2.fit(X_train2, y_train2)

In [ ]:
# Evaluar el modelo en los datos de prueba
linreg_y_pred1 = linreg_H1.predict(X_test1)
linreg_y_pred2 = linreg_H2.predict(X_test2)

In [ ]:
# Calcular el coeficiente de determinación del modelo
print("Coeficiente de determinación del modelo regresión lineal en H1: %0.20f" %r2_score(y_test1, linreg_y_pred1))
print("Coeficiente de determinación del modelo regresión lineal en H2: %0.20f" % r2_score(y_test2, linreg_y_pred2))

In [ ]:
# graficar la línea de regresión y los puntos de datos
plt.scatter(y_test1, linreg_y_pred1)
plt.plot([y_test1.min(), y_test1.max()], [y_test1.min(), y_test1.max()], 'k--', lw=2)
plt.xlabel('Valor real')
plt.ylabel('Predicción')
plt.title('Precisión del modelo lineal (R1 = %0.20f)' % r2_score(y_test1, linreg_y_pred1))
plt.show()

In [ ]:
# graficar la línea de regresión y los puntos de datos
plt.scatter(y_test2, linreg_y_pred2)
plt.plot([y_test2.min(), y_test2.max()], [y_test2.min(), y_test2.max()], 'k--', lw=2)
plt.xlabel('Valor real')
plt.ylabel('Predicción')
plt.title('Precisión del modelo lineal (R2 = %0.20f)' % r2_score(y_test2, linreg_y_pred2))
plt.show()

In [ ]:
# Calculando el MSE
print("MSE (Error cuadrático medio) del modelo regresión lineal en H1: %0.20f" % mean_squared_error(y_test1, linreg_y_pred1))
print("MSE (Error cuadrático medio) del modelo regresión lineal en H2: %0.20f" % mean_squared_error(y_test2, linreg_y_pred2))

In [ ]:
# calcular el MSE para diferentes tamaños del conjunto de datos de prueba
mse_values_linreg = []
sizes_linreg = range(1, len(y_test1) + 1)
for s in sizes_linreg:
    mse_values_linreg.append(mean_squared_error(y_test1[:s], linreg_y_pred1[:s]))

# graficar el MSE
plt.plot(sizes_linreg, mse_values_linreg)
plt.title('MSE del modelo lineal H1')
plt.xlabel('Tamaño del conjunto de prueba')
plt.ylabel('MSE')
plt.show()

In [ ]:
# calcular el MSE para diferentes tamaños del conjunto de datos de prueba
mse_values_linreg = []
sizes_linreg = range(1, len(y_test2) + 1)
for s in sizes_linreg:
    mse_values_linreg.append(mean_squared_error(y_test2[:s], linreg_y_pred2[:s]))

# graficar el MSE
plt.plot(sizes_linreg, mse_values_linreg)
plt.title('MSE del modelo lineal H2')
plt.xlabel('Tamaño del conjunto de prueba')
plt.ylabel('MSE')
plt.show()

# IMPLEMENTACIÓN DEL MÉTODO SAFE

### MODELO DE BOSQUE ALEATORIO SUPERVISOR FLEXIBLE

In [ ]:
pip install safe-transformer

In [ ]:
from SafeTransformer import SafeTransformer
# Seleccionar las columnas numéricas para transformar
numeric_cols = ['LeadTime','IsRepeatedGuest','PreviousBookingsNotCanceled','BookingChanges','DaysInWaitingList','RequiredCarParkingSpaces','TotalOfSpecialRequests']

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train1_SAFE, X_test1_SAFE, y_train1_SAFE, y_test1_SAFE = train_test_split(h1[(numeric_cols)], h1['IsCanceled'], test_size=0.2, random_state=42)
X_train2_SAFE, X_test2_SAFE, y_train2_SAFE, y_test2_SAFE = train_test_split(h2[(numeric_cols)], h2['IsCanceled'], test_size=0.2, random_state=42)

In [ ]:
# Entrenar un modelo complejo (bosque aleatorio supervisor flexible)
surrogate_model_H1 = RandomForestClassifier(n_estimators=100,
                                          max_depth=4,
                                          max_features='sqrt',
                                          random_state=42)
surrogate_model_H1 = surrogate_model_H1.fit(X_train1_SAFE, y_train1_SAFE)

In [ ]:
# Entrenar un modelo complejo (bosque aleatorio supervisor flexible)
surrogate_model_H2 = RandomForestClassifier(n_estimators=100,
                                          max_depth=4,
                                          max_features='sqrt',
                                          random_state=42)
surrogate_model_H2 = surrogate_model_H2.fit(X_train2_SAFE, y_train2_SAFE)

### TRANSFORMACIÓN DE VARIABLES

In [ ]:
# Utilizar SAFE para encontrar transformaciones de variables
safe_transformer_H1 = SafeTransformer(surrogate_model_H1)
X_transformed_H1 = safe_transformer_H1.fit_transform(X_train1_SAFE, y_train1_SAFE)

In [ ]:
safe_transformer_H2 = SafeTransformer(surrogate_model_H2)
X_transformed_H2 = safe_transformer_H2.fit_transform(X_train2_SAFE, y_train2_SAFE)

In [ ]:
# Realizar selección de características en el nuevo conjunto de características
linear_model_H1 = LogisticRegression(solver='lbfgs')
pipe_H1 = Pipeline(steps=[('linear', linear_model_H1)])
pipe_H1 = pipe_H1.fit(X_transformed_H1, y_train1_SAFE)

In [ ]:
linear_model_H2 = LogisticRegression(solver='lbfgs')
pipe_H2 = Pipeline(steps=[('linear', linear_model_H2)])
pipe_H2 = pipe_H2.fit(X_transformed_H2, y_train2_SAFE)

In [ ]:
# Evaluar el modelo de regresión logística basado en las variables transformadas
X_test_transformed_H1 = safe_transformer_H1.transform(X_test1_SAFE)
predictions_H1 = pipe_H1.predict(X_test_transformed_H1)
accuracy_H1 = accuracy_score(y_test1_SAFE, predictions_H1)

In [ ]:
X_test_transformed_H2 = safe_transformer_H2.transform(X_test2_SAFE)
predictions_H2 = pipe_H2.predict(X_test_transformed_H2)
accuracy_H2 = accuracy_score(y_test2_SAFE, predictions_H2)

In [ ]:
# Calculando el MSE
print("MSE (Error cuadrático medio) del modelo de Bosque Aleatorio en H1: %0.20f" % mean_squared_error(y_test1_SAFE, predictions_H1))
print("MSE (Error cuadrático medio) del modelo de Bosque Aleatorio en H2: %0.20f" % mean_squared_error(y_test2_SAFE, predictions_H2))

In [ ]:
# Calculando el R2
print("Coeficiente de determinación del modelo en Bosque Aleatorio H1: %0.20f" % r2_score(y_test1_SAFE, predictions_H1))
print("Coeficiente de determinación del modelo en Bosque Aleatorio H2: %0.20f" % r2_score(y_test2_SAFE, predictions_H2))

In [ ]:
# Calcular la precisión del modelo
print("Precisión del modelo Logístico en H1: %0.20f" % accuracy_score(y_test1_SAFE, predictions_H1))
print("Precisión del modelo Logístico en H2: %0.20f" % accuracy_score(y_test2_SAFE, predictions_H2))

# EVALUACIÓN CON EL MODELO DE REGRESIÓN LOGÍSTICA 

In [ ]:
# Entrenar un modelo de regresión logística con las variables transformadas
logistic_model_H1 = LogisticRegression(penalty='l2', C=0.1, solver='liblinear')
LM_H1 = logistic_model_H1.fit(X_transformed_H1, y_train1_SAFE)

In [ ]:
logistic_model_H2 = LogisticRegression(penalty='l2', C=0.1, solver='liblinear')
LM_H2 = logistic_model_H2.fit(X_transformed_H2, y_train2_SAFE)

In [ ]:
# Evaluar el modelo en el conjunto de prueba
y_pred_H1 = LM_H1.predict(X_test_transformed_H1)
y_pred_H2 = LM_H2.predict(X_test_transformed_H2)

In [ ]:
# Calcular el coeficiente de determinación del modelo
print("Coeficiente de determinación del modelo Logístico en H1: %0.20f" % r2_score(y_test1_SAFE, y_pred_H1))
print("Coeficiente de determinación del modelo Logístico en H2: %0.20f" % r2_score(y_test2_SAFE, y_pred_H2))

In [ ]:
# graficar la línea de regresión y los puntos de datos
plt.scatter(y_test1_SAFE, y_pred_H1)
plt.plot([y_test1_SAFE.min(), y_test1_SAFE.max()], [y_test1_SAFE.min(), y_test1_SAFE.max()], 'k--', lw=2)
plt.xlabel('Valor real')
plt.ylabel('Predicción')
plt.title('Precisión del modelo lineal (R1 = %0.20f)' % r2_score(y_test1_SAFE, y_pred_H1))
plt.show()

In [ ]:
# graficar la línea de regresión y los puntos de datos
plt.scatter(y_test2_SAFE, y_pred_H2)
plt.plot([y_test2_SAFE.min(), y_test2_SAFE.max()], [y_test2_SAFE.min(), y_test2_SAFE.max()], 'k--', lw=2)
plt.xlabel('Valor real')
plt.ylabel('Predicción')
plt.title('Precisión del modelo lineal (R2 = %0.20f)' % r2_score(y_test2_SAFE, y_pred_H2))
plt.show()

In [ ]:
# Calculando el MSE
print("MSE (Error cuadrático medio) del modelo Logístico en H1: %0.20f" % mean_squared_error(y_test1_SAFE, y_pred_H1))
print("MSE (Error cuadrático medio) del modelo Logístico en H2: %0.20f" % mean_squared_error(y_test2_SAFE, y_pred_H2))

In [ ]:
# calcular el MSE para diferentes tamaños del conjunto de datos de prueba
mse_values_logistic_model = []
sizes_logistic_model = range(1, len(y_test1_SAFE) + 1)
for s in sizes_logistic_model:
    mse_values_logistic_model.append(mean_squared_error(y_test1_SAFE[:s], y_pred_H1[:s]))

# graficar el MSE
plt.plot(sizes_logistic_model, mse_values_logistic_model)
plt.title('MSE del modelo lineal H1')
plt.xlabel('Tamaño del conjunto de prueba')
plt.ylabel('MSE')
plt.show()

In [ ]:
# calcular el MSE para diferentes tamaños del conjunto de datos de prueba
mse_values_logistic_model = []
sizes_logistic_model = range(1, len(y_test2_SAFE) + 1)
for s in sizes_logistic_model:
    mse_values_logistic_model.append(mean_squared_error(y_test2_SAFE[:s], y_pred_H2[:s]))

# graficar el MSE
plt.plot(sizes_logistic_model, mse_values_logistic_model)
plt.title('MSE del modelo lineal H2')
plt.xlabel('Tamaño del conjunto de prueba')
plt.ylabel('MSE')
plt.show()

In [ ]:
# Calcular la precisión del modelo
print("Precisión del modelo Logístico en H1: %0.20f" % accuracy_score(y_test1_SAFE, y_pred_H1))
print("Precisión del modelo Logístico en H2: %0.20f" % accuracy_score(y_test2_SAFE, y_pred_H2))